In [1]:
import os
import re
import collections
import numpy as py
import pandas as pd
import itertools

from ic_functions_py3 import*

In [2]:
# Create dictionaries of all associated names with the FlyBase gene symbol
geneID,NametoIDs,NametoFBid = geneIDdictionary('Datasets/fbgn_annotation_ID_fb_2016_05.tsv')

# Read in BDGP insitu data
Stage = ['1_3','4_6','7_8','9_10','11_12','13_16']
insitu = pd.read_csv('Datasets/insitu_annot.csv',names=['gene_symb','CG','FBgn','stage','staining'])
insitu_ns = insitu[insitu.staining != 'no staining']
insitu.head(5)

,gene_symb,CG,FBgn,stage,staining
0,a10,CG6642,FBgn0011293,1,no staining
1,a10,CG6642,FBgn0011293,2,no staining
2,a10,CG6642,FBgn0011293,3,no staining
3,a10,CG6642,FBgn0011293,4,no staining
4,a10,CG6642,FBgn0011293,5,no staining


In [3]:
# Read in all datasets
# Cross-referencing dictionary for promoters
crossref = pd.read_csv('Datasets/EPD/cross_references.txt',header=0,sep='\t')

# Promoters (-50 to +50)
raw_promoters = readFASTA('Datasets/EPD/dm6promoters_minus50plus50.fa','sequences',0,geneID,re.compile('>FP\d+\s([\w\-\(\)\.]+_\d)'))
promoters = {}
title = []
neither = []
for k,v in raw_promoters.items():
    number = k[-2:]
    if k[:-2] in geneID.keys(): 
        promoters[geneID[k[:-2]][0]+number] = v
    elif k[:-2].lower() in geneID.keys(): 
        promoters[geneID[k[:-2].lower()][0]+number] = v
    elif k[:-2].title() in geneID.keys(): 
        promoters[geneID[k[:-2].title()][0]+number] = v
        title.append(k[:-2].title())
    elif not crossref['Ensembl Gene ID'][crossref['Associated Gene Name'] == k[:-2]].empty:
        FBgn = crossref['Ensembl Gene ID'][crossref['Associated Gene Name'] == k[:-2]].values[0]
        if FBgn in geneID.keys():
            promoters[geneID[FBgn][0]+number] = v
        else:
            print('%s is in the cross references but not in geneIDs' %FBgn)
    else:
        neither.append(k)
        # note: these genes seem to be withdrawn, in progress, or not found in FlyBase
# Note: Both FBgn0283709 and FBgn0267825 are unannotated
od_promoters = collections.OrderedDict(sorted(promoters.items()))

FBgn0283709 is in the cross references but not in geneIDs
FBgn0267825 is in the cross references but not in geneIDs


In [5]:
# Read in Vienna Tile enhancers
VT = pd.read_csv('Datasets/Stark_ViennaTile/nature13395-s2/2014-01-00083C-Supplementary Table 4.csv')
VT.head(5)

,VTID,Chromosome,Start,End,FlyBase ID,Symbol,Chromosome.1,Start.1,End.1,Orientation,Match
0,VT0006,chr2L,16836,18924,FBgn0002121,l(2)gl,chr2L,9839,21376,-,2
1,VT0025,chr2L,54047,56104,FBgn0051973,Cda5,chr2L,25402,65404,-,2
2,VT0131,chr2L,285217,287380,FBgn0031245,CG3625,chr2L,283385,291795,-,2
3,VT0132,chr2L,288875,290893,FBgn0025686,Amnionless,chr2L,287252,289144,-,2
4,VT0132,chr2L,288875,290893,FBgn0031245,CG3625,chr2L,283385,291795,-,1


In [5]:
# Read in CAD/RedFly "known" enhancers
Known = pd.read_excel('Datasets/Furlong_4C/nature13417/nature13417-s3.xlsx')
Known.head(5)

,Source,Name,Chr,Start,Stop,assigned gene,FBGn
0,REDFly,ci_7.1,4,76479,83352,ci,FBgn0004859
1,CAD,ey_UE2.0,4,720530,722315,ey,FBgn0005558
2,CAD,ey_UE0.9,4,724596,725290,ey,FBgn0005558
3,CAD,ey_DO2,4,730630,730841,ey,FBgn0005558
4,REDFly,sphinx_1067bp_5'_fragment,4,994776,995842,sphinx,FBgn0083990


In [6]:
# Read in Furlong 4C enhancers
Furlong4C = pd.read_excel('Datasets/Furlong_4C/nature13417/nature13417-s2.xlsx')
Furlong4C.head(5)

,view,cond,pvalue,chr,start,end,midpoint,mid_dist,max_dist,short_dist,zscore,feature_id,feature
0,CRM_1088,MESO_3-4h,0.001,chr2R,15164485,15165376,15164930,-135286,-135286,-134840,4.194707,NaN,intragenic
1,CRM_1088,MESO_3-4h,0.001,chr2R,15182513,15183814,15183164,-117052,-117052,-116402,4.234489,NaN,intragenic
2,CRM_1088,MESO_3-4h,0.001,chr2R,15191016,15191709,15191362,-108854,-108854,-108507,3.489552,NaN,intragenic
3,CRM_1088,MESO_3-4h,0.001,chr2R,15197589,15198942,15198266,-101950,-101950,-101274,3.378263,FBgn0034433,promoter
4,CRM_1088,MESO_3-4h,0.001,chr2R,15206172,15206872,15206522,-93694,-93694,-93344,3.572291,NaN,intragenic


In [7]:
# Find core promoter motifs
_,_,filenames = next(os.walk('matrix'),(None, None, []))
pmotif_list = [x[:-4] for x in filenames if '.txt' in x]
print(pmotif_list)

od_len_promoter = createSeqIN(od_promoters,0,0,os.getcwd(),'promoters',1,0)
runPatser(os.getcwd(),'promoters',pmotif_list,0)

['BREd', 'BREu', 'Bridge', 'DInr', 'DPE', 'DTCT', 'MTE', 'TATABox', 'XCPE1', 'XCPE2']
Done: BREd
Done: BREu
Done: Bridge
Done: DInr
Done: DPE
Done: DTCT
Done: MTE
Done: TATABox
Done: XCPE1
Done: XCPE2


In [4]:
# Run patser once
_,_,filenames = next(os.walk('matrix'),(None, None, []))
pmotif_list = [x[:-4] for x in filenames if '.txt' in x]
print(pmotif_list)

motif_range = {}
motif_range['DInr'] = (-10,10)
motif_range['TATABox'] = (-40,-20)
motif_range['DPE'] = (20,40)
motif_range['MTE'] = (10,30)
motif_range['DTCT'] = (-10,10)

lnp_cutoff = {k:0 for k in pmotif_list}
promoters_patser_output = parsePatser(os.getcwd(),pmotif_list,'promoters',lnp_cutoff,'promoters',0,'.txt')
promoters_patser_output.head(5)

['BREd', 'BREu', 'Bridge', 'DInr', 'DPE', 'DTCT', 'MTE', 'TATABox', 'XCPE1', 'XCPE2']
BREd
BREu
Bridge
DInr
DPE
DTCT
MTE
TATABox
XCPE1
XCPE2


,sequence,motif,position,complement,score,lnp
0,128up_1,BREd,5,0,1.14,-2.57
1,128up_1,BREd,6,0,0.92,-2.29
2,128up_1,BREd,7,0,2.37,-4.76
3,128up_1,BREd,35,0,0.23,-1.56
4,128up_1,BREd,37,0,0.22,-1.56


In [4]:
# Read in patser output
_,_,filenames = next(os.walk('matrix'),(None, None, []))
pmotif_list = [x[:-4] for x in filenames if '.txt' in x]
print(pmotif_list)

motif_range = {}
motif_range['DInr'] = (-10,10)
motif_range['TATABox'] = (-40,-20)
motif_range['DPE'] = (20,40)
motif_range['MTE'] = (10,30)
motif_range['DTCT'] = (-10,10)

promoters_patser_output = pd.read_csv('patserAnalysis/promoters_patser_output.txt',header=None,sep='\t')
promoters_patser_output.columns = ['sequence','motif','position','complement','score','lnp']
promoters_patser_output[['position','complement']] = promoters_patser_output[['position','complement']].astype(int)
promoters_patser_output[['score','lnp']] = promoters_patser_output[['score','lnp']].astype(float)

promoters_patser_output.head(5)

['BREd', 'BREu', 'Bridge1', 'Bridge2', 'DInr', 'DPE', 'DTCT', 'MTE', 'TATABox', 'XCPE1', 'XCPE2']


,sequence,motif,position,complement,score,lnp
0,128up_1,BREd,4,0,0.00,-1.38
1,128up_1,BREd,5,0,1.14,-2.57
2,128up_1,BREd,6,0,0.92,-2.29
3,128up_1,BREd,7,0,2.37,-4.76
4,128up_1,BREd,35,0,0.23,-1.56


In [5]:
ES_cutoff=dict(zip(pmotif_list,[0.5,0.05,0.01,0.01,0.01,0.01,0.1,0.01,0.01,0.01,0.01]))
ES_cutoff

{'BREd': 0.5,
 'BREu': 0.05,
 'Bridge1': 0.01,
 'Bridge2': 0.01,
 'DInr': 0.01,
 'DPE': 0.01,
 'DTCT': 0.1,
 'MTE': 0.01,
 'TATABox': 0.01,
 'XCPE1': 0.01,
 'XCPE2': 0.01}

In [68]:
pNseq=dict(zip(pmotif_list,[41,22,2,43,43,43,71,25,1,113,52]))

In [ ]:
# Score all possible sequences using ElemeNT algorithm
nt2idx=dict(zip(['A','C','G','T'],[0,1,2,3]))
columns = ['motif', 'sequence', 'score']
ElemeNT = pd.DataFrame(data=np.zeros((0,len(columns))), columns=columns) 
for k,v in pPFM.items():
    L = v.shape[0]
    for seq in itertools.combinations(['A','T','C','G']*L,L):
        temp_score = 0
        for i,base in enumerate(seq):
            temp_score = temp_score*v[i,nt2idx[base]]/max(v[i,:])
            if temp_score == 0:
                ElemeNT = [k,''.join(seq),temp_score]
                continue
        ElemeNT = [k,seq,temp_score]
        print(temp_score)

In [45]:
nt2idx=dict(zip(['A','C','G','T'],[0,1,2,3]))
idx2nt={v:k for k,v in nt2idx.items()}
seq_constraints = {}
for k,v in pPFM.items():
    temp_nt = []
    for i,row in enumerate(v):
        temp_position = ''
        if 0 in row:
            for j,element in enumerate(row):
                if element > 0:
                    temp_position += idx2nt[j]
        else:
            temp_position = 'ACGT'
        temp_nt.append(temp_position)
        seq_constraints[k] = temp_nt
# seq_constraints = {k:v for k,v in seq_constraints.items() if not all(elem == 'ACGT' for elem in v)}
seq_constraints

{'BREd': ['ACGT', 'ACGT', 'ACGT', 'ACGT', 'ACGT', 'ACGT', 'ACGT'],
 'BREu': ['ACGT', 'ACGT', 'ACGT', 'ACGT', 'ACGT', 'ACGT', 'ACGT'],
 'Bridge1': ['ACGT', 'ACGT', 'ACGT', 'ACGT', 'ACGT'],
 'Bridge2': ['ACGT', 'ACGT', 'ACGT', 'ACGT'],
 'DInr': ['ACGT', 'ACGT', 'A', 'ACGT', 'ACGT', 'ACGT'],
 'DPE': ['ACGT', 'CG', 'ACGT', 'ACGT', 'ACGT', 'ACGT'],
 'DTCT': ['ACGT', 'ACGT', 'ACGT', 'ACGT', 'ACGT', 'ACGT', 'ACGT', 'ACGT'],
 'MTE': ['ACGT',
  'ACGT',
  'ACGT',
  'ACGT',
  'ACGT',
  'ACGT',
  'ACGT',
  'ACGT',
  'ACGT',
  'ACGT',
  'ACGT',
  'ACGT'],
 'TATABox': ['T', 'A', 'T', 'A', 'ACGT', 'ACGT', 'ACGT', 'ACGT'],
 'XCPE1': ['ACGT',
  'ACGT',
  'ACGT',
  'ACGT',
  'ACGT',
  'ACGT',
  'ACGT',
  'ACGT',
  'ACGT',
  'ACGT'],
 'XCPE2': ['ACGT',
  'ACGT',
  'ACGT',
  'ACGT',
  'ACGT',
  'ACGT',
  'ACGT',
  'ACGT',
  'ACGT',
  'ACGT',
  'ACGT']}

In [46]:
nt2idx=dict(zip(['A','C','G','T'],[0,1,2,3]))
columns = ['motif', 'sequence', 'score']
ElemeNT = pd.DataFrame(data=np.zeros((0,len(columns))), columns=columns) 

# k = 'TATABox'
# v = pPFM['TATABox']
idx = 0
for k,v in pPFM.items():
    L = v.shape[0]
    for seq in itertools.combinations(['A','T','C','G']*L,L):
        if all(elem in seq_constraints[k][i] for i,elem in enumerate(seq)):
            for i,base in enumerate(seq):
                if i == 0:
                    temp_score = v[i,nt2idx[base]]/max(v[i,:])
                else:
                    temp_score = temp_score*v[i,nt2idx[base]]/max(v[i,:])
                if temp_score == 0:
                    continue
            ElemeNT.loc[idx] = [k,''.join(seq),temp_score]
            idx += 1

KeyboardInterrupt: 

In [114]:
ElemeNT.head(5)

,motif,sequence,score
0,TATABox,ATCGATCG,0.0
1,TATABox,ATCGATCA,0.0
2,TATABox,ATCGATCT,0.0
3,TATABox,ATCGATCC,0.0
4,TATABox,ATCGATCG,0.0


In [ ]:
columns = ['sequence', 'motif', 'position','score']
filtered_ppatser_output = pd.DataFrame(data=np.zeros((0,len(columns))), columns=columns)
idx2 = 0
for idx in range(promoters_patser_output.shape[0]):
    temp = promoters_patser_output.loc[idx]
    if promoters[temp.position:temp.position+pPFM[temp.motif].shape[0]] in ElemeNT.sequence[ElemeNT.motif==temp.motif]:
        filtered_ppatser_output.loc[idx2] = [temp.sequence,temp.motif,temp.position,ElemeNT.score[(ElemeNT.sequence==temp.sequence) & (ElemeNT.motif==temp.motif)]]

In [14]:
def ElemeNT_scoring(sequence_dictionary,motif_list,PFM_dictionary,score_cutoff):
    bases = ['A','C','G','T']
    base2index = dict([reversed(x) for x in enumerate(bases)])
    
    output = pd.DataFrame(columns=['sequence','motif','position','score'])
    with open('element_output.txt','w') as f:
        for motif in motif_list:
            for seq_name, sequence in sequence_dictionary.items():
                for i in range(len(sequence)-PFM_dictionary[motif].shape[0]+1):
                    subsequence = sequence[i:i+PFM_dictionary[motif].shape[0]]
                    if 'N' in subsequence:
                        continue
                    temp_score = 1
                    for position, base in enumerate(subsequence):
                        temp_score = temp_score*PFM_dictionary[motif][position,base2index[base]]/max(PFM_dictionary[motif][position,:])
                    if np.round(temp_score,2) > score_cutoff[motif]:
                        f.write('%s\t%s\t%s\t%.4f\n' %(seq_name.replace('-','_'), motif, i+1, temp_score))
    output = pd.read_csv('element_output.txt',sep='\t',header=None)
    output.columns = ['sequence','motif','position','score']
    output[['score']] = output[['score']].astype(float)
    
    return output

In [15]:
ElemeNT_scoring({'test':'GGAGC'},['Bridge1'],pPFM,ES_cutoff)

,sequence,motif,position,score
0,test,Bridge1,1,0.435


In [16]:
ppatser_output = ElemeNT_scoring(promoters,pmotif_list,pPFM,ES_cutoff)

In [17]:
ppatser_output.head(5)

,sequence,motif,position,score
0,ECSIT_1,BREd,71,0.6065
1,Fhos_4,BREd,32,0.9317
2,CG33552_1,BREd,58,0.5349
3,CG33552_1,BREd,61,0.7859
4,mrt_2,BREd,63,0.5518
